# OpenAI Word Embeddings, Semantic Search

Word embeddings are a way of representing words and phrases as vectors. They can be used for a variety of tasks, including semantic search, anomaly detection, and classification. Via OpenAI Whisper, here is how words whose vectors are numerically similar are also similar in semantic meaning. In this tutorial, we will learn how to implement semantic search using embeddings woth various models. Understanding the Embeddings concept will be crucial to the next several videos in this series since we will use it to build several practical applications.

https://colab.research.google.com/drive/1BeB8FSmN2GED5qqqoWsxhByq6VlMwvGC


# Read Data File Containing Words

Let's fetch below nvidia-earnings-call as example

In [ ]:
import pandas as pd
import numpy as np
import requests

url = "https://gist.githubusercontent.com/hackingthemarkets/e664894b65b31cbe8993e02d25d26768/raw/618afe09d07979cc72911ce79634ab5d2cc19a54/nvidia-earnings-call.txt"
response = requests.get(url)
transcript = response.text

transcript

"Operator\n\nGood afternoon. My name is Emma, and I will be your conference operator today. At this time, I would like to welcome everyone to the NVIDIA's third quarter earnings call. [Operator instructions] Simona Jankowski, you may begin your conference.\n\nSimona Jankowski -- Vice President, Investor Relations\n\nThank you. Good afternoon, everyone, and welcome to NVIDIA's conference call for the third quarter of fiscal 2023. With me today from NVIDIA are Jen-Hsun Huang, president and chief executive officer; and Colette Kress, executive vice president and chief financial officer. I'd like to remind you that our call is being webcast live on NVIDIA's investor relations website.\n\n\nThe webcast will be available for replay until the conference call to discuss our financial results for the fourth quarter and fiscal 2023. The content of today's call is NVIDIA's property. It can't be reproduced or transcribed without our prior written consent. During this call, we may make forward-look

In [ ]:
words = transcript.split(" ")
len(words)

7912

In [ ]:
sentences = transcript.split(".")
len(sentences)

483

# Calculate Word Embeddings

To use word embeddings for semantic search, you first compute the embeddings for a corpus of text using a word embedding algorithm. What does this mean? We are going to create a numerical representation of each of these words. To perform this computation, we'll use OpenAI's 'get_embedding' function.

Since we have our words in a pandas dataframe, we can use "apply" to apply the get_embedding function to each row in the dataframe. We then store the calculated word embeddings in a new text file called "word_embeddings.csv" so that we don't have to call OpenAI again to perform these calculations.

In [ ]:
!pip install openai -q

import openai
from getpass import getpass
openai.api_key = getpass()
get_embedding("the fox crossed the road", engine='text-embedding-ada-002')

# get word embedding
df = transcript.split(" ")
print(df)

from openai.embeddings_utils import get_embedding
df['embedding'] = df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
df.to_csv('word_embeddings.csv')

KeyboardInterrupt: ignored

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.9 MB/s eta 0:00:00


**get_avg_embedding_bartbase** Get average embedding for a sentence using Bart

In [4]:
from transformers import BartTokenizer, BartModel
import torch
def get_avg_embedding_bartbase(sentence):

    model_name = 'facebook/bart-base'
    model = BartModel.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

    # Get the model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Retrieve the last hidden state of the encoder
    last_hidden_state = outputs.last_hidden_state

    # Average the embeddings across the tokens to get a single sentence-level embedding
    avg_embedding = torch.mean(last_hidden_state, dim=1).squeeze()

    return avg_embedding

In [6]:
sentence = "hi"
embeddings = get_avg_embedding_bartbase(sentence)
embeddings

tensor([ 9.1868e-01,  7.7761e-01,  9.5644e-01,  3.9279e+00,  6.2265e-01,
         9.2581e-01,  3.8260e-01, -1.1908e+00, -5.8801e-01, -8.5330e-01,
         1.0866e-01,  1.8332e+00,  7.6544e-01,  9.2052e-01,  2.5039e-01,
         5.6201e-03, -3.4683e-02,  8.4623e-01,  5.3869e-01, -9.2236e-01,
        -9.3307e-02, -1.3940e+00,  1.2814e+00,  2.3899e+00,  7.4410e-01,
        -9.3955e-01,  1.0515e+00, -1.4713e+01,  8.2986e-01, -3.9822e+00,
         7.7689e-01, -8.1913e-01, -2.0213e-02, -1.3128e+00, -2.5356e+00,
         7.2731e-01,  5.7129e-01,  1.8054e+00,  3.7047e-01, -1.1405e+00,
         5.6987e-01, -3.2628e+00, -1.3467e-01, -1.0481e-01, -2.9303e-01,
         9.3390e-01, -2.0538e+00,  8.4153e-01, -6.4223e-01,  4.4328e-01,
         3.8974e-01,  3.9185e-01, -1.8484e-01, -3.8155e-02, -1.2025e+00,
        -3.8610e-01,  1.0013e+00, -1.6833e+00,  8.6945e-01,  5.7616e-01,
        -2.4704e-02,  9.8925e-01, -5.6645e-01,  4.7827e-01, -4.0573e-01,
         7.3177e-01, -2.1424e+00, -8.5498e-01, -1.1

In [ ]:
sentences

['Operator\n\nGood afternoon',
 ' My name is Emma, and I will be your conference operator today',
 " At this time, I would like to welcome everyone to the NVIDIA's third quarter earnings call",
 ' [Operator instructions] Simona Jankowski, you may begin your conference',
 '\n\nSimona Jankowski -- Vice President, Investor Relations\n\nThank you',
 " Good afternoon, everyone, and welcome to NVIDIA's conference call for the third quarter of fiscal 2023",
 ' With me today from NVIDIA are Jen-Hsun Huang, president and chief executive officer; and Colette Kress, executive vice president and chief financial officer',
 " I'd like to remind you that our call is being webcast live on NVIDIA's investor relations website",
 '\n\n\nThe webcast will be available for replay until the conference call to discuss our financial results for the fourth quarter and fiscal 2023',
 " The content of today's call is NVIDIA's property",
 " It can't be reproduced or transcribed without our prior written consent",


In [ ]:
import time
start_time = time.time()
df = pd.DataFrame()
df['sentence'] = sentences
df['embedding'] = df['sentence'].apply(lambda x: get_avg_embedding_bartbase(x))
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 1571.20 seconds


In [ ]:
search_term = input('Enter a search term: ')

Enter a search term: demand for NVIDIA next quarter


In [ ]:
search_term

'demand for NVIDIA next quarter'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       483 non-null    object
 1   embedding  483 non-null    object
dtypes: object(2)
memory usage: 7.7+ KB


In [ ]:
from scipy.spatial.distance import cosine
search_term_vector = get_embeddings(search_term)
df["similarities"] = df['embedding'].apply(lambda x: 1- cosine(x, search_term_vector))

In [ ]:
df.sort_values("similarities", ascending=False).head(5)

,text,embedding,similarities
22,We delivered record data center and automotiv...,"[tensor(0.0628), tensor(0.4024), tensor(-0.362...",0.632423
52,\n\nNetworking posted strong growth driven by ...,"[tensor(0.7528), tensor(0.4230), tensor(-0.551...",0.625981
91,These dynamics are expected to continue in Q4,"[tensor(0.1341), tensor(-0.1798), tensor(-1.02...",0.624680
147,5 billion a quarter,"[tensor(0.1014), tensor(0.2658), tensor(-1.152...",0.622923
266,\n\nYou said gaming resumed sequential growth,"[tensor(1.1551), tensor(0.9886), tensor(0.5139...",0.616457


# Semantic Search

Now that we have our word embeddings stored, let's load them into a new dataframe and use it for semantic search. Since the 'embedding' in the CSV is stored as a string, we'll use apply() and to interpret this string as Python code and convert it to a numpy array so that we can perform calculations on it.

In [ ]:
df = pd.read_csv('word_embeddings.csv')
df['embedding'] = df['embedding'].apply(eval).apply(np.array)
df

FileNotFoundError: ignored

Let's now prompt ourselves for a search term that isn't in the dataframe. We'll use word embeddings to perform a semantic search for the words that are most similar to the word we entered. I'll first try the word "hot dog". Then we'll come back and try the word "yellow".

In [ ]:
search_term = input('Enter a search term: ')


Enter a search term: hot dog


Now that we have a search term, let's calculate an embedding or vector for that search term using the OpenAI get_embedding function.

In [ ]:
# semantic search
search_term_vector = get_embedding(search_term, engine="text-embedding-ada-002")
search_term_vector

[-0.021754980087280273,
 -0.00431280629709363,
 -0.0095869405195117,
 -0.0313945971429348,
 -0.021504772827029228,
 0.003371231956407428,
 -0.027839001268148422,
 -0.013708798214793205,
 -0.009435499086976051,
 -0.025692474097013474,
 0.017501434311270714,
 0.03186867758631706,
 -0.006597605999559164,
 0.004655197262763977,
 -0.0196347925812006,
 0.010429748333990574,
 0.04032309353351593,
 -0.010864321142435074,
 0.028392093256115913,
 0.00474737910553813,
 -0.021570617333054543,
 0.0036444859579205513,
 0.011200128123164177,
 -0.01926606334745884,
 -0.002057636622339487,
 0.0014979594852775335,
 0.010179540142416954,
 -0.005425576586276293,
 -0.004019799176603556,
 -0.020543444901704788,
 0.041850682348012924,
 0.010976256802678108,
 -0.022439762949943542,
 -0.000737868482246995,
 -0.0058634416200220585,
 -0.012530184350907803,
 -0.0007872518035583198,
 -0.002612375421449542,
 0.009343316778540611,
 0.00021728643332608044,
 0.0028708146419376135,
 -0.006008299067616463,
 0.0032313126

 Once we have a vector representing that word, we can see how similar it is to other words in our dataframe by calculating the cosine similarity of our search term's word vector to each word embedding in our dataframe.

In [ ]:
from openai.embeddings_utils import cosine_similarity

df["similarities"] = df['embedding'].apply(lambda x: cosine_similarity(x, search_term_vector))

df

,Unnamed: 0,text,embedding,similarities
0,0,red,"[1.8579006791696884e-05, -0.024676261469721794...",0.812070
1,1,potatoes,"[0.005025846417993307, -0.031079445034265518, ...",0.816856
2,2,soda,"[0.025859493762254715, -0.007452284451574087, ...",0.820797
3,3,cheese,"[-0.003942061681300402, -0.009351087734103203,...",0.824127
4,4,water,"[0.019031280651688576, -0.01257743313908577, 0...",0.798268
5,5,blue,"[0.005434895399957895, -0.0072994716465473175,...",0.786934
6,6,crispy,"[-0.0010056837927550077, -0.005415474995970726...",0.820502
7,7,hamburger,"[-0.013206875883042812, -0.0018223668448626995...",0.876765
8,8,coffee,"[-0.0007566261338070035, -0.0194522924721241, ...",0.799683
9,9,green,"[0.01538460049778223, -0.010931522585451603, 0...",0.785477


# Sorting By Similarity

Now that we have calculated the similarities to each term in our dataframe, we simply sort the similarity values to find the terms that are most similar to the term we searched for. Notice how the foods are most similar to "hot dog". Not only that, it puts fast food closer to hot dog. Also some colors are ranked closer to hot dog than others. Let's go back and try the word "yellow" and walk through the results.

In [ ]:
df.sort_values("similarities", ascending=False).head(20)

,Unnamed: 0,text,embedding,similarities
7,7,hamburger,"[-0.013206875883042812, -0.0018223668448626995...",0.876765
18,18,cheeseburger,"[-0.018216600641608238, 0.005054354667663574, ...",0.856907
14,14,french fries,"[0.0014476682990789413, -0.016491735354065895,...",0.838477
3,3,cheese,"[-0.003942061681300402, -0.009351087734103203,...",0.824127
2,2,soda,"[0.025859493762254715, -0.007452284451574087, ...",0.820797
6,6,crispy,"[-0.0010056837927550077, -0.005415474995970726...",0.820502
1,1,potatoes,"[0.005025846417993307, -0.031079445034265518, ...",0.816856
13,13,chocolate,"[0.0015591585543006659, -0.013005273416638374,...",0.816746
0,0,red,"[1.8579006791696884e-05, -0.024676261469721794...",0.812070
16,16,cake,"[-0.013669422827661037, -0.016827937215566635,...",0.811998


# Adding Words Together

What's even more interesting is that we can add word vectors together. What happens when we add the numbers for milk and espresso, then search for the word vector most similar to milk + espresso? Let's make a copy of the original dataframe and call it food_df. We'll operate on this copy. Let's try adding word together. Let's add milk + espresso and store the results in milk_espresso_vector.

In [ ]:
food_df = df.copy()

milk_vector = food_df['embedding'][10]
espresso_vector = food_df['embedding'][19]

milk_espresso_vector = milk_vector + espresso_vector
milk_espresso_vector

array([-0.02157659, -0.03206679, -0.01620988, ..., -0.00423221,
        0.00078145, -0.02898556])

Now let's find the words most similar to milk + espresso. If you have never done this before, it's pretty surprising that you can add words together like this and find similar words using numbers.

In [ ]:
food_df["similarities"] = food_df['embedding'].apply(lambda x: cosine_similarity(x, milk_espresso_vector))
food_df.sort_values("similarities", ascending=False)

,Unnamed: 0,text,embedding,similarities
19,19,espresso,"[-0.02250584401190281, -0.012747502885758877, ...",0.960501
10,10,milk,"[0.0009292512550018728, -0.019319288432598114,...",0.960501
15,15,latte,"[-0.015634099021553993, -0.003942839801311493,...",0.922975
22,22,mocha,"[-0.012487593106925488, -0.026140518486499786,...",0.899327
8,8,coffee,"[-0.0007566261338070035, -0.0194522924721241, ...",0.895382
3,3,cheese,"[-0.003942061681300402, -0.009351087734103203,...",0.885276
13,13,chocolate,"[0.0015591585543006659, -0.013005273416638374,...",0.883440
2,2,soda,"[0.025859493762254715, -0.007452284451574087, ...",0.874156
4,4,water,"[0.019031280651688576, -0.01257743313908577, 0...",0.866049
7,7,hamburger,"[-0.013206875883042812, -0.0018223668448626995...",0.852722


# Microsoft Earnings Call Transcript

Let's tie this back to finance. I have attached some text from a recent [Microsoft earnings call here](https://gist.github.com/hackingthemarkets/1c827a7750384fcf52c84594ef216a2d). Click on "raw" and save the file as a CSV. Upload it to Google Colab as microsoft-earnings.csv. Let's use what we just learned to perform a semantic search on sentences in the Microsoft earnings call. We'll start by reading the paragraphs into a pandas dataframe.

In [ ]:
earnings_df = pd.read_csv('microsoft-earnings.csv')
earnings_df

,text
0,"Thank you, Brett. To start, I want to outline ..."
1,"With that context, this quarter, the Microsoft..."
2,It helps them align their spend with demand an...
3,We are the platform of choice for customers' S...
4,Now to data and AI. With our Microsoft Intelli...
...,...
57,Other income and expense should be roughly $10...
58,"And finally, as a reminder, for Q2 cash flow, ..."
59,And FX should decrease COGS and operating expe...
60,With the high margins in our Windows OEM busin...


Once we have the dataframe, we'll once again compute the embeddings for each line in our CSV file.

In [ ]:
earnings_df['embedding'] = earnings_df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
earnings_df.to_csv('earnings-embeddings.csv')

If you download the earnings_embeddings.csv file locally and open it up, you'll see that our embeddings are for entire paragraphs - not just words. This means that we'll be able to search on similar sentences even if there isn't an exact match for the string we search for. We are searching on meaning.

In [ ]:
earnings_search = input("Search earnings for a sentence:")

Search earnings for a sentence:artificial intelligence demand cloud products


In [ ]:

earnings_search_vector = get_embedding(earnings_search, engine="text-embedding-ada-002")
earnings_search_vector

[-0.010817994363605976,
 -0.02560822106897831,
 -0.0023312214761972427,
 0.0036306483671069145,
 0.020452769473195076,
 0.027453476563096046,
 -0.02305866777896881,
 -0.00854311604052782,
 0.0005885277641937137,
 -0.03546836972236633,
 0.022776948288083076,
 0.022974152117967606,
 0.010162997990846634,
 -0.004535669460892677,
 -0.01735386624932289,
 0.0059794774278998375,
 0.02036825381219387,
 0.007099308539181948,
 0.0042891656048595905,
 -0.02156555838882923,
 -0.010691220872104168,
 0.025185642763972282,
 0.015184632502496243,
 -0.004394810181111097,
 -0.009655904956161976,
 -0.01467753853648901,
 0.02812960185110569,
 -0.036510732024908066,
 0.001288862549699843,
 0.011339173652231693,
 0.024255970492959023,
 -0.01767784170806408,
 -0.0077824764885008335,
 -0.005585071165114641,
 -0.010205255821347237,
 0.002774928230792284,
 -0.0033559727016836405,
 -0.012205458246171474,
 0.018734287470579147,
 -0.006662644911557436,
 0.016015702858567238,
 0.01366335153579712,
 0.01407184358686

In [ ]:

earnings_df["similarities"] = earnings_df['embedding'].apply(lambda x: cosine_similarity(x, earnings_search_vector))

earnings_df


,text,embedding,similarities
0,"Thank you, Brett. To start, I want to outline ...","[-0.009504559449851513, -0.003731543431058526,...",0.749671
1,"With that context, this quarter, the Microsoft...","[-0.0016425022622570395, -0.028921114280819893...",0.800683
2,It helps them align their spend with demand an...,"[0.008828130550682545, -0.03199512138962746, 0...",0.796945
3,We are the platform of choice for customers' S...,"[0.011994918808341026, -0.024179909378290176, ...",0.800598
4,Now to data and AI. With our Microsoft Intelli...,"[-0.004754434805363417, 0.0038801338523626328,...",0.820970
...,...,...,...
57,Other income and expense should be roughly $10...,"[-0.01832527294754982, -0.014160438440740108, ...",0.688968
58,"And finally, as a reminder, for Q2 cash flow, ...","[-0.012947804294526577, -0.010494815185666084,...",0.715620
59,And FX should decrease COGS and operating expe...,"[0.0009612650028429925, -0.01565629616379738, ...",0.771278
60,With the high margins in our Windows OEM busin...,"[0.010544494725763798, -0.03846913203597069, -...",0.757963


In [ ]:
earnings_df.sort_values("similarities", ascending=False)

,text,embedding,similarities
5,"Cosmos DB now supports postscript SQL, making ...","[-0.00441406574100256, -0.005979578942060471, ...",0.846190
12,Our cloud for sustainability is off to a fast ...,"[0.008922123350203037, -0.016263997182250023, ...",0.823616
4,Now to data and AI. With our Microsoft Intelli...,"[-0.004754434805363417, 0.0038801338523626328,...",0.820970
11,"All up more than 400,000 organizations now use...","[-0.001035509048961103, -0.020460907369852066,...",0.813934
9,Power Automate has more than seven million mon...,"[-0.025241363793611526, -0.034008558839559555,...",0.806075
...,...,...,...
29,"Thank you, Satya, and good afternoon, everyone...","[0.013616200536489487, -0.01455166470259428, -...",0.718449
44,Operating expenses increased 2% and 5% in cons...,"[0.017512913793325424, 0.008054900914430618, 0...",0.716947
58,"And finally, as a reminder, for Q2 cash flow, ...","[-0.012947804294526577, -0.010494815185666084,...",0.715620
57,Other income and expense should be roughly $10...,"[-0.01832527294754982, -0.014160438440740108, ...",0.688968


# Sentences of the Fed Speech

Let's use the Fed Speech example once more. Let's calculate the word embeddings for a particular sentence in the November 2nd speech that we discussed in the OpenAI Whisper tutorial. Then we'll take a new sentence from a future speech that isn't in our dataset, and find the most similar sentence in our dataset. Here is the sentence we will use to search for similarity:

"the inflation is too damn high"

As we did previously, take [the linked CSV file](https://gist.github.com/hackingthemarkets/9b55ea8b73c7f4e04b42a9f8eddb8393) and upload it to Google Colab as fed-speech.csv. We'll once again read it into a pandas dataframe.

In [ ]:
fed_df = pd.read_csv('fed-speech.csv')
fed_df

,text
0,Good afternoon
1,My colleagues and I are strongly committed to ...
2,We have both the tools that we need and the re...
3,Price stability is the responsibility of the F...
4,"Without price stability, the economy does not ..."
5,"In particular, without price stability, we wil..."
6,"Today, the FOMC raised our policy interest rat..."
7,We are moving our policy stance purposefully t...
8,"In addition, we are continuing the process of ..."
9,Restoring price stability will likely require ...


We'll once again calculate the embeddings and save them in a new CSV file.

In [ ]:
fed_df['embedding'] = fed_df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
fed_df.to_csv('fed-embeddings.csv')

We'll then enter the new sentence that we want to find similarity for:

"We will continue to increase interest rates and tighten monetary policy"

In [ ]:
fed_sentence = input('Enter something Jerome Powell said: ')


Enter something Jerome Powell said: the inflation is too damn high


Again we'll get the vector for this sentence, find the cosine similarity, and sort by most similar.

In [ ]:
fed_sentence_vector = get_embedding(fed_sentence, engine="text-embedding-ada-002")
fed_sentence_vector

[-0.00413066940382123,
 -0.011251280084252357,
 -0.005313646513968706,
 -0.02224256657063961,
 -0.012122263200581074,
 0.0024195776786655188,
 -0.03860924765467644,
 -0.005732887890189886,
 -0.016691673547029495,
 -0.0204096008092165,
 0.022372564300894737,
 0.006987363565713167,
 0.023464541882276535,
 0.006652620155364275,
 0.014026726596057415,
 0.011277279816567898,
 0.0338253416121006,
 0.007643850985914469,
 0.02031860314309597,
 -0.015677694231271744,
 0.0025706999003887177,
 0.011101783253252506,
 -0.0122522609308362,
 -0.0034319330006837845,
 -0.020214606076478958,
 -0.0012877873377874494,
 0.016340680420398712,
 -0.02594749443233013,
 -0.0051089003682136536,
 -0.002343204338103533,
 0.007513853255659342,
 -0.0077023496851325035,
 -0.03166738152503967,
 -0.0024634518194943666,
 -0.020019609481096268,
 -0.03564530611038208,
 -0.013870729133486748,
 -0.016990669071674347,
 -0.0031215641647577286,
 -0.00859933253377676,
 0.026168489828705788,
 -0.010932786390185356,
 0.0133507391

In [ ]:
fed_df = pd.read_csv('fed-embeddings.csv')
fed_df['embedding'] = fed_df['embedding'].apply(eval).apply(np.array)
fed_df


,Unnamed: 0,text,embedding
0,0,Good afternoon,"[-0.017524775117635727, 0.02069251798093319, -..."
1,1,My colleagues and I are strongly committed to ...,"[-0.026972517371177673, -0.012394015677273273,..."
2,2,We have both the tools that we need and the re...,"[0.003941578324884176, -0.015006175264716148, ..."
3,3,Price stability is the responsibility of the F...,"[0.009378707036376, -0.016561055555939674, -0...."
4,4,"Without price stability, the economy does not ...","[-0.003026996273547411, -0.014454687014222145,..."
5,5,"In particular, without price stability, we wil...","[-0.03618694841861725, -0.008898851461708546, ..."
6,6,"Today, the FOMC raised our policy interest rat...","[-0.024621201679110527, -0.02114815264940262, ..."
7,7,We are moving our policy stance purposefully t...,"[-0.025701606646180153, -0.012234759517014027,..."
8,8,"In addition, we are continuing the process of ...","[-0.03149143233895302, 0.0019273122306913137, ..."
9,9,Restoring price stability will likely require ...,"[-0.010953230783343315, -0.020290518179535866,..."


In [ ]:

fed_df["similarities"] = fed_df['embedding'].apply(lambda x: cosine_similarity(x, fed_sentence_vector))

fed_df


,Unnamed: 0,text,embedding,similarities
0,0,Good afternoon,"[-0.017524775117635727, 0.02069251798093319, -...",0.750047
1,1,My colleagues and I are strongly committed to ...,"[-0.026972517371177673, -0.012394015677273273,...",0.826724
2,2,We have both the tools that we need and the re...,"[0.003941578324884176, -0.015006175264716148, ...",0.770154
3,3,Price stability is the responsibility of the F...,"[0.009378707036376, -0.016561055555939674, -0....",0.775339
4,4,"Without price stability, the economy does not ...","[-0.003026996273547411, -0.014454687014222145,...",0.804080
5,5,"In particular, without price stability, we wil...","[-0.03618694841861725, -0.008898851461708546, ...",0.775005
6,6,"Today, the FOMC raised our policy interest rat...","[-0.024621201679110527, -0.02114815264940262, ...",0.787081
7,7,We are moving our policy stance purposefully t...,"[-0.025701606646180153, -0.012234759517014027,...",0.812895
8,8,"In addition, we are continuing the process of ...","[-0.03149143233895302, 0.0019273122306913137, ...",0.745955
9,9,Restoring price stability will likely require ...,"[-0.010953230783343315, -0.020290518179535866,...",0.790525


In [ ]:

fed_df.sort_values("similarities", ascending=False)

,Unnamed: 0,text,embedding,similarities
24,24,The recent inflation data again have come in h...,"[-0.021040253341197968, -0.009753845632076263,...",0.871317
22,22,Inflation remains well above our longer run go...,"[-0.023937253281474113, -0.0032772799022495747...",0.869225
31,31,My colleagues and I are acutely aware that hig...,"[-0.011414038017392159, -0.01515731681138277, ...",0.847498
29,29,The longer the current amount of high inflatio...,"[-0.018355058506131172, -0.012731979601085186,...",0.847374
32,32,We are highly attentive to the risks that high...,"[-0.025864068418741226, -0.015762366354465485,...",0.833747
27,27,"Despite elevated inflation, longer term inflat...","[-0.023557519540190697, -0.024205774068832397,...",0.828953
1,1,My colleagues and I are strongly committed to ...,"[-0.026972517371177673, -0.012394015677273273,...",0.826724
37,37,"It will take time, however, for the full effec...","[-0.02066067047417164, -0.018034202978014946, ...",0.826675
46,46,Reducing inflation is likely to require a sust...,"[-0.03423553332686424, -0.014666956849396229, ...",0.823727
26,26,Russia's war against Ukraine has boosted price...,"[-0.009621184319257736, -0.019101163372397423,...",0.818095


# Calculating Cosine Similarity

We used the Cosine Similarity function, but how does it actually work? Cosine similarity is just calculating the similarity between two vectors. There is a mathematical equation for calculating the angle between two vectors.

![](https://drive.google.com/uc?export=view&id=1cehvtx7LKuFeq_LqfnLi-gzIz1D1wSf9)

In [ ]:
v1 = np.array([1,2,3])
v2 = np.array([4,5,6])

# (1 * 4) + (2 * 5) + (3 * 6)
dot_product = np.dot(v1, v2)
dot_product

32

In [ ]:
# square root of (1^2 + 2^2 + 3^2) = square root of (1+4+9) = square root of 14
np.linalg.norm(v1)

3.7416573867739413

In [ ]:
# square root of (4^2 + 5^2 + 6^2) = square root of (16+25+36) = square root of 14
np.linalg.norm(v2)

8.774964387392123

In [ ]:
magnitude = np.linalg.norm(v1) * np.linalg.norm(v2)
magnitude

32.83291031876401

In [ ]:
dot_product / magnitude

0.9746318461970762

In [ ]:
from scipy import spatial

result = 1 - spatial.distance.cosine(v1, v2)

result

0.9746318461970761